# Project: Investigate a Dataset - Soccer Database

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 
Below is what to expect to find in this in the soccer dataset. I acquired this data from  [kaggle](https://www.kaggle.com/datasets/hugomathien/soccer).

Soccer dataset for data analysis:

- +25,000 matches
- +10,000 players 
- 11 European Countries with their lead championship
- Seasons 2008 to 2016
- Players and Teams' attributes* sourced from EA Sports' FIFA video game series, including the weekly updates
- Team line up with squad formation (X, Y coordinates)
- Betting odds from up to 10 providers
- Detailed match events (goal types, possession, corner, cross, fouls, cards etc…) for +10,000 matcheAbout Dataset

This dataset is stored in a SQLite database.



### Questions for Analysis
>- **What is the age distribution of players?**
>- **What is the most common formation?**
>- **What teams improved the most over the time period?**
>- **Which players had the most penalties?**
>- **What team attributes lead to the most victories?**
>- **What player attributes lead to the most victories?**
>- **How do the player attributes that lead to most victories change with time?**
>- **How does the potential of players over time?**


In [1]:
# I am importing all the packages I will need

import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Data Wrangling

I am using sqlite3 package to connect to the sqlite soccer database

In [2]:
conn = sqlite3.connect('database.sqlite')

In [3]:
cur = conn.cursor()

I am looking to see what are the available tables and their names

In [4]:
table_names = cur.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
for i,table in enumerate(table_names):
    print(i,table[0])

0 sqlite_sequence
1 Player_Attributes
2 Player
3 Match
4 League
5 Country
6 Team
7 Team_Attributes


**In the following cells I read in the data from the sqlite database into pandas dataframes for easy exploration and analysis**

### Player Attributes

As a non soccer fan I find I am lost trying to understand the player attriibutes.

I set out to the internet to find clarity and I found discription of these attributes [here](https://www.guidetofm.com/players/attributes/)

In [50]:
ply_attr = pd.read_sql(f'SELECT * FROM player_attributes', conn)

In [51]:
ply_attr.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [52]:
ply_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [53]:
ply_attr.describe()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


### Players

In [55]:
players = pd.read_sql(f'SELECT * FROM player', conn)

In [56]:
players.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [57]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  11060 non-null  int64  
 1   player_api_id       11060 non-null  int64  
 2   player_name         11060 non-null  object 
 3   player_fifa_api_id  11060 non-null  int64  
 4   birthday            11060 non-null  object 
 5   height              11060 non-null  float64
 6   weight              11060 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 605.0+ KB


I am looking to see if there are duplicated rows and remove them

In [58]:
sum(players.duplicated())

0

I am converting the players dataframe datatypes to appropriate types

- `birthday` to `datatime`
- `weight` to `float`

In [61]:
players.birthday = pd.to_datetime(players.birthday)

In [65]:
players.weight = players.weight.astype(float)

In [66]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  11060 non-null  int64         
 1   player_api_id       11060 non-null  int64         
 2   player_name         11060 non-null  object        
 3   player_fifa_api_id  11060 non-null  int64         
 4   birthday            11060 non-null  datetime64[ns]
 5   height              11060 non-null  float64       
 6   weight              11060 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 605.0+ KB


In [67]:
players.describe()

,id,player_api_id,player_fifa_api_id,height,weight
count,11060.000000,11060.000000,11060.000000,11060.000000,11060.000000
mean,5537.511392,156582.427215,165664.910488,181.867445,168.380289
std,3197.692647,160713.700624,58649.928360,6.369201,14.990217
min,1.000000,2625.000000,2.000000,157.480000,117.000000
25%,2767.750000,35555.500000,151889.500000,177.800000,159.000000
50%,5536.500000,96619.500000,184671.000000,182.880000,168.000000
75%,8306.250000,212470.500000,203883.250000,185.420000,179.000000
max,11075.000000,750584.000000,234141.000000,208.280000,243.000000


### Matches

In [68]:
matches = pd.read_sql(f"SELECT * FROM match", conn)

In [69]:
matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [70]:
matches.columns.values

array(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal', 'away_team_goal', 'home_player_X1',
       'home_player_X2', 'home_player_X3', 'home_player_X4',
       'home_player_X5', 'home_player_X6', 'home_player_X7',
       'home_player_X8', 'home_player_X9', 'home_player_X10',
       'home_player_X11', 'away_player_X1', 'away_player_X2',
       'away_player_X3', 'away_player_X4', 'away_player_X5',
       'away_player_X6', 'away_player_X7', 'away_player_X8',
       'away_player_X9', 'away_player_X10', 'away_player_X11',
       'home_player_Y1', 'home_player_Y2', 'home_player_Y3',
       'home_player_Y4', 'home_player_Y5', 'home_player_Y6',
       'home_player_Y7', 'home_player_Y8', 'home_player_Y9',
       'home_player_Y10', 'home_player_Y11', 'away_player_Y1',
       'away_player_Y2', 'away_player_Y3', 'away_player_Y4',
       'away_player_Y5', 'away_player_Y6', 'away_player_Y7',
       'aw

I am looking to see the datatypes of the matches dataframe

In [78]:
matches.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 115 columns):
 #    Column            Dtype  
---   ------            -----  
 0    id                int64  
 1    country_id        int64  
 2    league_id         int64  
 3    season            object 
 4    stage             int64  
 5    date              object 
 6    match_api_id      int64  
 7    home_team_api_id  int64  
 8    away_team_api_id  int64  
 9    home_team_goal    int64  
 10   away_team_goal    int64  
 11   home_player_X1    float64
 12   home_player_X2    float64
 13   home_player_X3    float64
 14   home_player_X4    float64
 15   home_player_X5    float64
 16   home_player_X6    float64
 17   home_player_X7    float64
 18   home_player_X8    float64
 19   home_player_X9    float64
 20   home_player_X10   float64
 21   home_player_X11   float64
 22   away_player_X1    float64
 23   away_player_X2    float64
 24   away_player_X3    float64
 25   away_player_X4    fl

In [74]:
sum(matches.duplicated())

0

below I will clean the matches data in these columns `goal, shoton, shotoff, foulcommit, card, cross, corner, possession`. The data is in the form of markdown format.

I will import the `BeautifulSoup` library so I can easily read the html structure

In [99]:
from bs4 import BeautifulSoup as bs

In [281]:
columns = ['match_api_id','goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession']

The cell below selects only the rows that are not `null` or `None`

In [280]:
html_format_matches_data = matches[columns][np.invert(matches[columns].isnull().any(axis=1))]

The functions below extracts information from the xml in columns and creates a new data frame of that column.

The new data frame will have the match id and desired clean data

In [393]:
def clean_html(dataframe, column_name):
    dataframe_subset = dataframe[['match_api_id',f'{column_name}']]
    
    if column_name=='goal':
        find = 'goals'
        
    elif column_name=='shoton':
        find = 'shoton'
        
    elif column_name=='shotoff':
        find = 'shotoff'
        
    elif column_name=='foulcommit':
        find = 'foulscommitted'
        
    elif column_name=='cross':
        find = 'crosses'
        
    elif column_name=='corner':
        find = 'corners'
        
    else:
        find='ycards'
   
    
    clean_arr = []
    for match_id, dirty_column in dataframe_subset.values:
        bs_content = bs(dirty_column, 'html.parser')
        
        for xml in bs_content.find_all('value'):
            try:
                player_id = xml.find('player1').get_text()
                team_id = xml.find('team').get_text()
                data = xml.find(find).get_text()
            except Exception:
                continue
            clean_arr.append([match_id, player_id, team_id, data])
            
    return pd.DataFrame(
        np.array(clean_arr), 
        columns=['match_api_id',f'player_api_id_{column_name}',f'team_api_id_{column_name}',f'{column_name}']
    )

In [462]:
def get_possession(dataframe):
    
    dataframe_subset = dataframe[['match_api_id','possession']]

    clean_arr = []
    for match_id, dirty_column in dataframe_subset.values:
        bs_content = bs(dirty_column, 'html.parser')
        
        for xml in bs_content.find_all('value'):
            try:
                away_pos = xml.find('awaypos').get_text()
                home_pos = xml.find('homepos').get_text()
            except Exception:
                continue
            clean_arr.append([match_id, home_pos, away_pos])
            
    return pd.DataFrame(
        np.array(clean_arr), 
        columns=['match_api_id','home_possession','away_possession']
    )

The below cells are extracting the data from the xml format.
<br>
The `clean_html` function extracts the matchID,playerID, teamID and the desired data then it produces and `:pandas: DataFrame` 
<br>
The data from these dataframes can easily be analysed and or joined together

In [394]:
goals_df = clean_html(html_format_matches_data, 'goal')

In [395]:
shoton_df = clean_html(html_format_matches_data, 'shoton')

In [396]:
shotoff_df = clean_html(html_format_matches_data, 'shotoff')

In [397]:
foulcommit_df = clean_html(html_format_matches_data, 'foulcommit')

In [398]:
card_df = clean_html(html_format_matches_data, 'card')

In [399]:
cross_df = clean_html(html_format_matches_data, 'cross')

In [400]:
corner_df = clean_html(html_format_matches_data, 'corner')

In [463]:
possession_df = get_possession(html_format_matches_data)

### **removing duplicates**

In [401]:
sum(goals_df.duplicated())

3657

In [402]:
goals_df.drop_duplicates(inplace=True)

In [403]:
goals_df = goals_df.astype(int)

#### <center>Shoton</center>

In [404]:
shoton_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56659 entries, 0 to 56658
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   match_api_id          56659 non-null  object
 1   player_api_id_shoton  56659 non-null  object
 2   team_api_id_shoton    56659 non-null  object
 3   shoton                56659 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


In [405]:
sum(shoton_df.duplicated())

12121

In [406]:
shoton_df.drop_duplicates(inplace=True)

In [407]:
sum(shoton_df.duplicated())

0

In [408]:
shoton_df = shoton_df.astype(int)

In [409]:
shoton_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44538 entries, 0 to 56658
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   match_api_id          44538 non-null  int64
 1   player_api_id_shoton  44538 non-null  int64
 2   team_api_id_shoton    44538 non-null  int64
 3   shoton                44538 non-null  int64
dtypes: int64(4)
memory usage: 1.7 MB


#### <center>Shotoff</center>

In [410]:
shotoff_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92944 entries, 0 to 92943
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   match_api_id           92944 non-null  object
 1   player_api_id_shotoff  92944 non-null  object
 2   team_api_id_shotoff    92944 non-null  object
 3   shotoff                92944 non-null  object
dtypes: object(4)
memory usage: 2.8+ MB


In [411]:
sum(shotoff_df.duplicated())

25593

In [412]:
shotoff_df.drop_duplicates(inplace=True)

In [413]:
shotoff_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67351 entries, 0 to 92943
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   match_api_id           67351 non-null  object
 1   player_api_id_shotoff  67351 non-null  object
 2   team_api_id_shotoff    67351 non-null  object
 3   shotoff                67351 non-null  object
dtypes: object(4)
memory usage: 2.6+ MB


In [414]:
shotoff_df = shotoff_df.astype(int)

In [415]:
shotoff_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67351 entries, 0 to 92943
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   match_api_id           67351 non-null  int64
 1   player_api_id_shotoff  67351 non-null  int64
 2   team_api_id_shotoff    67351 non-null  int64
 3   shotoff                67351 non-null  int64
dtypes: int64(4)
memory usage: 2.6 MB


#### <center>Foulcommit</center>

In [416]:
foulcommit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210144 entries, 0 to 210143
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   match_api_id              210144 non-null  object
 1   player_api_id_foulcommit  210144 non-null  object
 2   team_api_id_foulcommit    210144 non-null  object
 3   foulcommit                210144 non-null  object
dtypes: object(4)
memory usage: 6.4+ MB


In [417]:
sum(foulcommit_df.duplicated())

87227

In [418]:
foulcommit_df.drop_duplicates(inplace=True)

In [419]:
foulcommit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122917 entries, 0 to 210141
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   match_api_id              122917 non-null  object
 1   player_api_id_foulcommit  122917 non-null  object
 2   team_api_id_foulcommit    122917 non-null  object
 3   foulcommit                122917 non-null  object
dtypes: object(4)
memory usage: 4.7+ MB


In [420]:
foulcommit_df.head()

,match_api_id,player_api_id_foulcommit,team_api_id_foulcommit,foulcommit
0,489042,25518,10261,1
1,489042,30929,10261,1
2,489042,29581,10261,1
3,489042,30373,10260,1
5,489042,24157,10260,1


In [421]:
foulcommit_df[foulcommit_df.player_api_id_foulcommit=='Unknown player']

,match_api_id,player_api_id_foulcommit,team_api_id_foulcommit,foulcommit
121755,704611,Unknown player,9875,1


In [442]:
foulcommit_df.drop(index=121755, inplace=True)

The cell above drops a row with an Unknown player
<br>
I discovered this cell when I got an error trying to convert the dataframe datatypes to `int` types
<br>
I first determine the index of where the row witht the `'Unknown player'` is

In [443]:
foulcommit_df = foulcommit_df.astype(int)

In [444]:
foulcommit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122916 entries, 0 to 210141
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   match_api_id              122916 non-null  int64
 1   player_api_id_foulcommit  122916 non-null  int64
 2   team_api_id_foulcommit    122916 non-null  int64
 3   foulcommit                122916 non-null  int64
dtypes: int64(4)
memory usage: 4.7 MB


#### <center>Card</center>

In [422]:
card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58045 entries, 0 to 58044
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   match_api_id        58045 non-null  object
 1   player_api_id_card  58045 non-null  object
 2   team_api_id_card    58045 non-null  object
 3   card                58045 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [423]:
sum(card_df.duplicated())

9

In [424]:
card_df.drop_duplicates(inplace=True)

In [425]:
sum(card_df.duplicated())

0

In [445]:
card_df[card_df.player_api_id_card=='Unknown player']

,match_api_id,player_api_id_card,team_api_id_card,card
32359,1287076,Unknown player,8551,1
33796,1287001,Unknown player,8551,1
33896,1287022,Unknown player,8551,1


In [446]:
card_df.drop(index=[32359,33796,33896], inplace=True)

The cell above drops a row with an Unknown player
<br>
I discovered this cell when I got an error trying to convert the dataframe datatypes to `int` types
<br>
I first determine the index of where the row witht the `'Unknown player'` is

In [447]:
card_df = card_df.astype(int)

In [448]:
card_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58033 entries, 0 to 58044
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   match_api_id        58033 non-null  int64
 1   player_api_id_card  58033 non-null  int64
 2   team_api_id_card    58033 non-null  int64
 3   card                58033 non-null  int64
dtypes: int64(4)
memory usage: 2.2 MB


#### <center>Cross</center>

In [427]:
cross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231104 entries, 0 to 231103
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   match_api_id         231104 non-null  object
 1   player_api_id_cross  231104 non-null  object
 2   team_api_id_cross    231104 non-null  object
 3   cross                231104 non-null  object
dtypes: object(4)
memory usage: 7.1+ MB


In [428]:
sum(cross_df.duplicated())

130996

In [429]:
cross_df.drop_duplicates(inplace=True)

In [430]:
sum(cross_df.duplicated())

0

In [431]:
cross_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100108 entries, 0 to 231100
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   match_api_id         100108 non-null  object
 1   player_api_id_cross  100108 non-null  object
 2   team_api_id_cross    100108 non-null  object
 3   cross                100108 non-null  object
dtypes: object(4)
memory usage: 3.8+ MB


In [432]:
cross_df = cross_df.astype(int)

In [433]:
cross_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100108 entries, 0 to 231100
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   match_api_id         100108 non-null  int64
 1   player_api_id_cross  100108 non-null  int64
 2   team_api_id_cross    100108 non-null  int64
 3   cross                100108 non-null  int64
dtypes: int64(4)
memory usage: 3.8 MB


#### <center>Corner</center>

In [434]:
corner_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85376 entries, 0 to 85375
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   match_api_id          85376 non-null  object
 1   player_api_id_corner  85376 non-null  object
 2   team_api_id_corner    85376 non-null  object
 3   corner                85376 non-null  object
dtypes: object(4)
memory usage: 2.6+ MB


In [435]:
sum(corner_df.duplicated())

52481

In [436]:
corner_df.drop_duplicates(inplace=True)

In [437]:
sum(corner_df.duplicated())

0

In [438]:
corner_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32895 entries, 0 to 85369
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   match_api_id          32895 non-null  object
 1   player_api_id_corner  32895 non-null  object
 2   team_api_id_corner    32895 non-null  object
 3   corner                32895 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [439]:
corner_df = corner_df.astype(int)

In [440]:
corner_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32895 entries, 0 to 85369
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   match_api_id          32895 non-null  int64
 1   player_api_id_corner  32895 non-null  int64
 2   team_api_id_corner    32895 non-null  int64
 3   corner                32895 non-null  int64
dtypes: int64(4)
memory usage: 1.3 MB


#### <center>Possession</center>

In [465]:
possession_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34796 entries, 0 to 34795
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   match_api_id     34796 non-null  object
 1   home_possession  34796 non-null  object
 2   away_possession  34796 non-null  object
dtypes: object(3)
memory usage: 815.7+ KB


In [466]:
sum(possession_df.duplicated())

4870

In [467]:
possession_df.drop_duplicates(inplace=True)

In [468]:
possession_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29926 entries, 0 to 34795
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   match_api_id     29926 non-null  object
 1   home_possession  29926 non-null  object
 2   away_possession  29926 non-null  object
dtypes: object(3)
memory usage: 935.2+ KB


In [470]:
possession_df = possession_df.astype(int)

In [471]:
possession_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29926 entries, 0 to 34795
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   match_api_id     29926 non-null  int64
 1   home_possession  29926 non-null  int64
 2   away_possession  29926 non-null  int64
dtypes: int64(3)
memory usage: 935.2 KB


In [472]:
possession_df.describe()

,match_api_id,home_possession,away_possession
count,2.992600e+04,29926.000000,29926.000000
mean,1.369415e+06,51.816347,48.183653
std,5.477462e+05,10.206609,10.206609
min,4.890420e+05,2.000000,6.000000
25%,8.572730e+05,45.000000,42.000000
50%,1.474654e+06,52.000000,48.000000
75%,1.983420e+06,58.000000,55.000000
max,2.118418e+06,94.000000,98.000000


I have cleaned and extracted the data from its xml format. Each dataframe I created has columns with extensions that show how or where they were extracted. 
<br>
Example:
<br>
- The `corner_df` has columns like this `player_api_id_corner, team_api_id_corner`. This shows the specific team and player involved and it makes it easier for future table merges

I will drop the columns I have extracted information from the `matches` dataframe

In [487]:
matches.drop(columns[1:], axis=1, inplace=True)

### League

In [36]:
league = pd.read_sql(f'SELECT * FROM league LIMIT {data_limit}', conn)

In [37]:
league.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [38]:
league.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          11 non-null     int64 
 1   country_id  11 non-null     int64 
 2   name        11 non-null     object
dtypes: int64(2), object(1)
memory usage: 392.0+ bytes


### Countries

In [39]:
country = pd.read_sql(f'SELECT * FROM country LIMIT {data_limit}',conn)

In [40]:
country.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [41]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11 non-null     int64 
 1   name    11 non-null     object
dtypes: int64(1), object(1)
memory usage: 304.0+ bytes


### Teams

In [42]:
teams = pd.read_sql(f'SELECT * FROM team LIMIT {data_limit}', conn)

In [43]:
teams.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [44]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                299 non-null    int64  
 1   team_api_id       299 non-null    int64  
 2   team_fifa_api_id  288 non-null    float64
 3   team_long_name    299 non-null    object 
 4   team_short_name   299 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 11.8+ KB


### Team Attributes

In [45]:
team_attr = pd.read_sql(f'SELECT * FROM team_attributes LIMIT {data_limit}', conn)

In [46]:
team_attr.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [47]:
team_attr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1000 non-null   int64  
 1   team_fifa_api_id                1000 non-null   int64  
 2   team_api_id                     1000 non-null   int64  
 3   date                            1000 non-null   object 
 4   buildUpPlaySpeed                1000 non-null   int64  
 5   buildUpPlaySpeedClass           1000 non-null   object 
 6   buildUpPlayDribbling            342 non-null    float64
 7   buildUpPlayDribblingClass       1000 non-null   object 
 8   buildUpPlayPassing              1000 non-null   int64  
 9   buildUpPlayPassingClass         1000 non-null   object 
 10  buildUpPlayPositioningClass     1000 non-null   object 
 11  chanceCreationPassing           1000 non-null   int64  
 12  chanceCreationPassingClass      100

In [489]:
team_attr.describe()

,id,team_fifa_api_id,team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
count,1000.000000,1000.000000,1000.000000,1000.000000,342.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,19274.666000,10395.597000,52.765000,48.663743,48.779000,51.921000,53.447000,54.224000,46.015000,49.535000,52.367000
std,288.819436,40703.790739,15981.628143,11.396388,9.536829,10.753129,10.077913,11.037165,10.338216,10.349874,9.713142,9.552568
min,1.000000,1.000000,1773.000000,23.000000,24.000000,20.000000,21.000000,20.000000,22.000000,23.000000,24.000000,29.000000
25%,250.750000,83.000000,8384.000000,45.000000,42.000000,40.000000,47.000000,47.000000,48.000000,38.750000,44.000000,47.000000
50%,500.500000,485.000000,8658.000000,52.000000,49.000000,50.000000,52.000000,53.000000,53.000000,46.000000,48.000000,52.000000
75%,750.250000,1903.000000,9885.000000,62.000000,54.750000,55.000000,57.000000,62.000000,63.000000,52.000000,56.000000,59.000000
max,1000.000000,112513.000000,274581.000000,80.000000,77.000000,79.000000,80.000000,80.000000,80.000000,72.000000,72.000000,70.000000


## Exploratory data analysis

### What is the age distribution of players?

I will look a the age of the players